In [18]:
import json
from datetime import datetime,date,timedelta,time
import time as t
import pandas as pd
import os
    
contract_multiple = 5.0
def settle(output):
    df = pd.DataFrame(output)
    if not len(df):
        return 0.0
    buys = df[df.direction == 'Buy'].reset_index()
    sells = df[df.direction != 'Buy'].reset_index()
    return ((sells.price - buys.price) * contract_multiple).sum()

def read_file(path):
    datas = {}
    with open(path, 'r') as f:
        datas = json.load(f)
    orders = {}
    settle_timestamps = []
    datetime_from = date(2016, 12, 5)
    datetime_to = date(2017, 11, 23)
    for i in range((datetime_to - datetime_from).days + 1):
        day = datetime_from + timedelta(days=i)
        dt = datetime.combine(day, time(15, 0, 0))
        dt = dt + timedelta(hours=8)

        settle_timestamps.append(int(t.mktime(dt.timetuple()) * 1000))
    output = {
        'order_id' : [],
        'direction' : [],
        'datetime' : [],
        'price' : [],
        'qty' : []
    }
    result = {
        'date' : [],
        'pl': [],
    }
    for order in datas['datas']:
        timestamp = int(order['timestamp'])
        while timestamp > settle_timestamps[0]:
            dt = datetime.fromtimestamp(settle_timestamps[0] / 1000.0)
            #print('{}:{}'.format(dt.strftime('%Y-%m-%d'), settle(output)))
            result['date'].append(dt.strftime('%Y-%m-%d'))
            result['pl'].append(settle(output))
            #output = reset_output()
            del settle_timestamps[0]

        if order['type'] == 'limit_order':
            orders[order['order_id']] = order['direction']
        elif order['type'] == 'trade_order':
            order_id = order['order_id']
            assert order_id in orders, order_id
            for i in range(int(order['qty'])):
                output['order_id'].append(order_id)
                output['direction'].append(orders[order_id])
                output['datetime'].append(datetime.utcfromtimestamp(int(order['timestamp']) / 1000.0))
                output['price'].append(float(order['price']))
                output['qty'].append(1)
        else:
            pass
    dt = datetime.fromtimestamp(settle_timestamps[0] / 1000.0)
    result['date'].append(dt.strftime('%Y-%m-%d'))
    result['pl'].append(settle(output))
    return result,output
    
path = 'd:/WorkSpace/geek_quant/backtesting/result/bechmark'
dfs = []
#for file in os.listdir(path):
#    result = read_file(os.path.join(path, file))
#    df = pd.DataFrame(result)
#    df['instrument'] = file
#    dfs.append(df)
#df = pd.concat(dfs)
result, output = read_file(os.path.join(path, 'zn1801.json'))
pd.DataFrame(result)

,date,pl
0,2016-12-05,0.0
1,2016-12-06,0.0
2,2016-12-07,0.0
3,2016-12-08,0.0
4,2016-12-09,0.0
5,2016-12-10,0.0
6,2016-12-11,0.0
7,2016-12-12,0.0
8,2016-12-13,0.0
9,2016-12-14,0.0


In [16]:
import pandas as pd

df = pd.read_csv('d:/portfolio/result.csv')
df = df[df.instrument == 'zn1801']
df[df.date > '2017-10-23']

,id,date,long_position,long_unfill,pl,short_position,short_unfill,instrument
39970,322,2017-10-23 15:00:00,0,0,0.0,0,0,zn1801
39971,323,2017-10-24 15:00:00,0,0,-1250.0,0,0,zn1801
39972,324,2017-10-25 15:00:00,0,0,-2125.0,0,0,zn1801
39973,325,2017-10-26 15:00:00,0,0,-2125.0,0,0,zn1801
39974,326,2017-10-27 15:00:00,0,0,-2125.0,1,0,zn1801
39975,327,2017-10-28 15:00:00,0,0,-1125.0,2,0,zn1801
39976,328,2017-10-29 15:00:00,0,0,-1125.0,2,0,zn1801
39977,329,2017-10-30 15:00:00,0,0,-3500.0,0,0,zn1801
39978,330,2017-10-31 15:00:00,0,0,-4225.0,0,0,zn1801
39979,331,2017-11-01 15:00:00,0,0,-4225.0,0,0,zn1801
